In [194]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import pathlib
from tqdm import tqdm
from queue import Empty
from datetime import datetime

L'obiettivo dello script è ottenere un ranking delle squadre in modo tale da aiutare per la classificazione. 
Questo ranking è dato dalla media dalle posizioni al termine del campoionato nelle ultime stagioni.

Voglio operare in questo modo: 
- prendere le squadre nella stagione scelta e ordinarli secondo il nome
- prendere la classifica degli anni precedenti e calcolare il rank per ciascuna stagione (il rank è dato dal numero di squadra - la posizione in classifica)
- se una squadra è neopromossa avrà un rank equivalente a 0 (quindi la squadra non c'è in altre stagioni)

In [195]:
def calculate_links(season):
    years = season.split("-")
    dictio = {}
    for i in range(10):
        y_l = int(years[0])-i
        y_r = int(years[1])-i
        season = str(y_l)+'-'+str(y_r)
        link = "https://fbref.com/it/comp/11/"+ season +"/Statistiche-di-Serie-A-"+ season
        dictio[season] = link
        
    return dictio

In [196]:
headers = {'User-Agent': 
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

model_season = '2021-2022' #stagione che uso nel modello
seasons = calculate_links(model_season) #ottengo i link delle 5 stagioni precedenti

In [197]:
pageTree = requests.get(seasons[model_season])
soup = BeautifulSoup(pageTree.text, features="lxml")

In [198]:
table = soup.select('table.stats_table')[0]
a_hrefs = table.find_all('a')
teams = [a_href for a_href in a_hrefs if 'it/squadre/' in str(a_href)]        

names_of_teams = sorted([name.contents[0] for name in teams])
names_of_teams

dataframe = pd.DataFrame()
dataframe['Team'] = names_of_teams

In [199]:
seasons.pop(model_season) #elimino la stagione che sto analizzando

'https://fbref.com/it/comp/11/2021-2022/Statistiche-di-Serie-A-2021-2022'

Vado a calcolare le posizioni in classifica nelle stagioni precedenti e creo nel dataframe una nuova colonna con la posizione in quella stagione.
Ovviamente le squadre che sono neopromosse, se non ci sono nelle stagioni precedenti, il valore sarà 0 di default 

In [200]:
for key_season, value_season in seasons.items():
    dataframe[key_season] = 0
    rank = len(names_of_teams)

    pageTree = requests.get(value_season)
    soup = BeautifulSoup(pageTree.text, features="lxml")

    table = soup.select('table.stats_table')[0]
    a_hrefs = table.find_all('a')
    teams = [a_href for a_href in a_hrefs if 'it/squadre/' in str(a_href)]        
    names_of_teams_in_season = [name.contents[0] for name in teams]

    for name in names_of_teams_in_season:
        index = dataframe.index[dataframe['Team'] == name].tolist()
        if index != []:
            dataframe.iat[index[0], dataframe.columns.get_loc(key_season)] = rank 
        rank -= 1


Qui calcolo la media per calcolare il Rank finale

In [201]:
for i, row in dataframe.iterrows():
    cols_seasons = row.drop('Team')
    avg = cols_seasons.mean()
    dataframe.loc[i, 'Rank'] = avg

In [206]:
dataframe.sort_values(by='Rank', ascending=False).to_csv("SerieA/Season21_22/rank.csv")